In [7]:
import os
import shutil
import pandas as pd
import cx_Oracle
import re
import sys
import time
import subprocess
import string

## 1. copy and paste

### 1.1 Defined functions

In [20]:
# Function to get current year and month from user
def get_year_month():
    # Define year and month pattern
    year_pattern = r"^\d{4}$"  # Pattern for four digits
    month_pattern = r"^(0[1-9]|1[0-2])$"  # Pattern for months 01-12

    while True:
        year = input("Enter the current year (yyyy): ")
        if re.match(year_pattern, year):
            break  # Exit loop if year is valid
        else:
            print("Invalid year format. Please enter the year again in 'yyyy' format.")

    while True:
        month = input("Enter the current month (mm): ")
        if re.match(month_pattern, month):
            break  # Exit loop if month is valid
        else:
            print("Invalid month format. Please enter the month again in 'mm' format (01-12).")

    return year, month

# Funtion to excute sql in python to check whether SYNC is updated 
def oracle_sql(period):
    print('===================== SYNC database check operation started... =====================')
    # configure the Oracle client. Solution reference: https://stackoverflow.com/questions/56119490/cx-oracle-error-dpi-1047-cannot-locate-a-64-bit-oracle-client-library
    lib_dir = r"C:\Users\u1158100\Desktop\oracleinstantclient_21_13\instantclient-basic-windows.x64-21.13.0.0.0dbru\instantclient_21_13"
    
    try:
    # Try to get the Oracle client version
        version = cx_Oracle.clientversion()
        print("Oracle is connected. Oracle client version:", version)
    except cx_Oracle.DatabaseError as e:
        print("Oracle is not connected. Error message shows:", e, ". Will try to initialize Oracle client.")
        try:
        # Try to initialize the Oracle client with the specified library directory
            cx_Oracle.init_oracle_client(lib_dir=lib_dir)
            print("Oracle client initialized successfully. Oracle client version:", cx_Oracle.clientversion())
        except cx_Oracle.DatabaseError as e:
            print("Failed to initialize Oracle client. Error message shows", e)
            sys.exit("Exiting program due to Oracle client initialization failure.")

    # configure the Oracle connection
    conf="IMS_DATA_SHARE/IMS_DATA_SHARE@shauorl001p.internal.imsglobal.com:1521/IMSORA.internal.imsglobal.com"
    sql = f"""
    SELECT COUNT(1)
    FROM IMS_APPLI.SYNC_FACT_ALL_INFO
    WHERE SALE_DATE = TO_DATE(:period, 'YYYYMM')
    ORDER BY TO_CHAR(SALE_DATE, 'YYYYMM')
    """
    try:
        conn = cx_Oracle.connect(conf) 
        cur = conn.cursor()
        cur.execute(sql, [period])
        count = cur.fetchone()[0]
        print("IMS_APPLI.SYNC_FACT_ALL_INFO databesed successfully updated.")
        print(f"Count of data for {period} is: {count}")
    except cx_Oracle.DatabaseError as e:
        print("SQL fails. Please check IMS_APPLI.SYNC_FACT_ALL_INFO. Error message shows:", e)
        sys.exit()  # Exit the program
    finally:
    # Close the cursor and connection to release resources
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()
    print('===================== SYNC database check operation ended. =====================\n')

# Function to create folder if it doesn't exist
def create_folder(path):
    print('===================== Folder creation operation started... =====================')
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder '{path}' created.")
    else:
        print(f"Folder '{path}' already exists.")
    print('===================== Folder creation operation ended. =====================\n')

# Function to copy files excluding 'product_molecule.csv'; src: source, dst: destination
def copy_files(src, dst):
    print('===================== Copy and paste operation started... =====================')

    # List to hold the names of duplicated files
    duplicated_files = [f for f in os.listdir(src) if os.path.isfile(os.path.join(dst, f)) and f != 'product_molecule.csv']
    
    # If there are duplicated files, ask the user what to do
    while duplicated_files:
        print("===================== The following files already exist in the destination directory: =====================")
        for filename in duplicated_files:
            print(filename)
        print("===================== Duplicate Files List End. =====================")

        user_input = input("Do you want to remove the existing files? (Y/N): ").strip().upper()
        if user_input == 'Y':
            print("===================== Start removing existing files... =====================")
            for filename in duplicated_files:
                os.remove(os.path.join(dst, filename))
                print(f"'{filename}' Removed.")
            print("===================== All existing files removed. =====================")
            break  # Exit the loop and proceed with copying files
        
        elif user_input == 'N':
            user_input = input("Do you want to continue to the RENAME operation? (Y/N): ").strip().upper()
            if user_input == 'Y':
                print("===================== Existing files will be overwitten. =====================")
                break

            elif user_input == 'N':
                print("===================== Exiting program... =====================")
                sys.exit()  # Exit the program
            else:
                print("Input format wrong. Please try again.")
                continue  # Ask the question again
        else:
            print("Input format wrong. Please try again.")

    print("===================== Start copying files... =====================")
    # Copy the files
    for filename in os.listdir(src):
        if filename != 'product_molecule.csv':
            src_file = os.path.join(src, filename)
            dst_file = os.path.join(dst, filename)
            shutil.copy2(src_file, dst_file)
            print(f"File '{filename}' copied.")
    print("===================== All files copied. =====================")
    print('===================== Copy and paste operation ended. =====================\n')

# Function to rename 'product_molecule_updated.csv' to 'product_molecule.csv'
def rename_file(path):
    print('===================== File renaming operation started... =====================')
    original = os.path.join(path, 'product_molecule_updated.csv')
    changed = os.path.join(path, 'product_molecule.csv')
    if os.path.exists(changed):
        user_input = input("The file 'product_molecule.csv' already exists. Do you want to overwrite it? (Y/N): ").strip().upper()
        if user_input == 'Y':
            os.remove(changed)
            print(f"'{changed}' Removed.")
        elif user_input == 'N':
            print("Exiting program...")
            sys.exit()
        else:
            print("Input format wrong. Please try again.")

    if os.path.exists(original):
        os.rename(original, changed)
        print(f"File '{original}' renamed to '{changed}'.")
    else:
        print(f"File '{original}' does not exist.")
    print('===================== File renaming operation ended. =====================\n')

# Function to replace "" with " " in 'pack.csv'
def replace_text_in_file(path, search_text, replace_text):
    file_path = os.path.join(path, 'pack.csv')
    with fileinput.FileInput(file_path, inplace=True) as file:
        for line in file:
            print(line.replace(search_text, replace_text), end='')

### 1.2 Main

In [21]:
# Main script execution
if __name__ == "__main__":
    print('===================== CHPA CR PD files pre-processing started... =====================')
    year, month = get_year_month()
    period = year + month
    print('Current period is:', period)
    
    # Network drive setup

    # Check whether SYNC is updated
    oracle_sql(period)

    # For test, all folder will be create in the project directory
    cr_directory = os.getcwd()
    folder_path = os.path.join(cr_directory, period)
    source_path = os.path.join(cr_directory, f'{month}M{year[2:]}')
    print('Current python source file directory is:', cr_directory)
    print('DS China raw data path is:', folder_path)
    print('PD source path is:', source_path)
    time.sleep(2)

    # Create folder
    create_folder(folder_path)
    time.sleep(2)   

    # Copy files
    source_path = os.path.join(cr_directory, f'{month}M{year[2:]}')
    copy_files(source_path, folder_path)
    time.sleep(2) 
    
    # Rename file
    rename_file(folder_path)

    # # Replace text in file
    # replace_text_in_file(folder_path, '""', '" "')

Current period is: 202312
===================== SYNC database check operation started... =====================
Oracle is connected. Oracle client version: (21, 13, 0, 0, 0)
IMS_APPLI.SYNC_FACT_ALL_INFO databesed successfully updated.
Count of data for 202312 is: 1498228
===================== SYNC database check operation ended. =====================

===================== Folder creation operation started... =====================
Folder 'c:\Users\u1158100\Desktop\CHPA_CR\CR_step0_automation\202312' already exists.
===================== Folder creation operation ended. =====================

c:\Users\u1158100\Desktop\CHPA_CR\CR_step0_automation\12M23
===================== Copy and paste operation started... =====================
===================== The following files already exist in the destination directory: =====================
cudu.csv
CUQ_vertical_simple.csv
lab.csv
manuflg.csv
ndf.CSV
pack.csv
product.csv
===================== Duplicate Files List End. =====================
==

## 2. Network Drive Setup

In [10]:
year='2023'
month='12'
period=year+month
PD_netdrive_dir = r"P:\\"
DC_netdrive_dir = r"Z:\SVNpro\Customized Report\MSD.Regional\RawData"

PD_work_dir = os.path.join(PD_netdrive_dir, f'{month}M{year[2:]}')
DC_work_dir = os.path.join(DC_netdrive_dir, period)

if os.path.exists(PD_work_dir):
    print('PD_work_dir exists')

if os.path.exists(DC_work_dir):
    print('DC_work_dir exists')


PD_work_dir exists
DC_work_dir exists


In [ ]:
# Define the paths for PD and DC network drives
pd_folder_dir = r'\\bejsfps02\Feng Mingzhu\hospital\MSD project'
cd_folder_dir = r'\\shasts004p\IA_Hospital\SVNpro\Customized Report\MSD.Regional\RawData'

def check_network_drives(): 
    if os.path.exists(pd_folder_dir) and os.path.exists(cd_folder_dir):
        print('PD, DC network already exist')
        return True
    else:
        user_input = input('Do you want to manually connect PD, DC network? (Y/N): ').strip().upper()
        if user_input == 'Y':
            print('Exiting program...')
            sys.exit()
        elif user_input == 'N':
            return False
        else:
            print('Invalid input, please enter Y or N.')
            return False

def get_available_drive_letters():
    # Get all available drive letters excluding the ones already in use
    drives = ['{}:'.format(d) for d in string.ascii_uppercase if not os.path.exists('{}:'.format(d))]
    print('Available drive letters:', drives)
    return drives[:2]  # Return the first two available letters

def connect_network_drive():
    # Pretend to connect the network drives (this is just a simulation)
    available_letters = get_available_drive_letters()
    pd_letter = available_letters[0]
    dc_letter = available_letters[1]
    
    print(f'PD network drive is connected using letter {pd_letter} and the specific path {PD_DRIVE_PATH}')
    print(f'DC network drive is connected using letter {dc_letter} and the specific path {DC_DRIVE_PATH}')


In [10]:
year='2023'
month='12'
period=year+month
pd_folder_dir = r'P:\\'
dc_folder_dir = r'Z:\SVNpro\Customized Report\MSD.Regional\RawData'
# sas_usage_dir = r'\\shasts004p\IA_Hospital\SVNpro\GDG\GDG每月使用情况'
# if os.path.exists(pd_folder_dir) and os.path.exists(dc_folder_dir):
#         print('PD, DC network already exist')
# else:
#     user_input = input('Do you want to manually connect PD, DC network? (Y/N): ').strip().upper()
#     if user_input == 'Y':
#         print('Exiting program...')
#         sys.exit()
#     elif user_input == 'N':
#          print('Test-N')
#     else:
#         print('Invalid input, please enter Y or N.')

print(os.listdir(os.path.join(pd_folder_dir, '12M23')))

['cudu.csv', 'CUQ_vertical_simple.csv', 'lab.csv', 'manuflg.csv', 'ndf.CSV', 'pack.csv', 'product.csv', 'product_molecule.csv', 'product_molecule_updated.csv']


In [5]:
os.makedirs(os.path.join(dc_folder_dir, '202401_test'))